<a href="https://colab.research.google.com/github/RohitPU/Hexart.In-Task3/blob/main/Butterfly_Images_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UPLOADING DATASET

In [2]:
with open('/content/kaggle.json', 'w') as f:
  f.write('{"username":"rohitpokalkar","key":"1b57df9d5b60cfd2a3ac238292be9958"}')

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d gpiosenka/butterfly-images40-species

100% 452M/454M [00:15<00:00, 39.4MB/s]
100% 454M/454M [00:15<00:00, 30.6MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/butterfly-images40-species.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

IMPORTING LIBRARIES

In [6]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

TRAINING DATA

In [8]:
train_dir = '/content/train'
validation_dir = '/content/valid'

class_names = []
for subdir in os.listdir(train_dir):
    class_names.append(subdir)
    
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
print(f'no.of classes: {nb_classes}')

nmb_train = 0
for subdir in os.listdir(train_dir):
    for filename in os.listdir(os.path.join(train_dir,subdir)):
        nmb_train +=1
        
nmb_val = 0
for subdir in os.listdir(validation_dir):
    for filename in os.listdir(os.path.join(validation_dir,subdir)):
        nmb_val +=1

no.of classes: 100


In [9]:
print(f'number of training images: {nmb_train}')
print(f'number of validation images: {nmb_val}')

number of training images: 12639
number of validation images: 500


In [10]:
from keras.preprocessing.image import ImageDataGenerator

GENERATORS

In [11]:
train_generator = ImageDataGenerator(rescale = 1./255.)

test_generator = ImageDataGenerator( rescale = 1./255. )

valid_generator = ImageDataGenerator( rescale = 1./255. )

train = train_generator.flow_from_directory(train_dir, batch_size = 20, class_mode = 'categorical', classes=class_names, target_size = (256, 256))

valid = valid_generator.flow_from_directory( validation_dir,  batch_size = 20, class_mode = 'categorical',classes=class_names, target_size = (256, 256))


Found 12639 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


CNN MODEL

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.softmax)
])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 123008)            0         
                                                                 
 dense (Dense)               (None, 128)               1

ACCURACY

In [14]:
history = model.fit(train, validation_data = valid,epochs = 50, batch_size=32)

Epoch 1/50
632/632 [==============================] - 42s 54ms/step - loss: 3.6638 - accuracy: 0.1850 - val_loss: 2.4998 - val_accuracy: 0.3920
Epoch 2/50
632/632 [==============================] - 33s 53ms/step - loss: 1.7034 - accuracy: 0.5805 - val_loss: 2.1283 - val_accuracy: 0.4740
Epoch 3/50
632/632 [==============================] - 33s 52ms/step - loss: 0.4181 - accuracy: 0.8928 - val_loss: 3.0056 - val_accuracy: 0.4620
Epoch 4/50
632/632 [==============================] - 33s 52ms/step - loss: 0.1105 - accuracy: 0.9745 - val_loss: 3.4889 - val_accuracy: 0.4420
Epoch 5/50
632/632 [==============================] - 32s 51ms/step - loss: 0.0677 - accuracy: 0.9867 - val_loss: 3.9425 - val_accuracy: 0.4380
Epoch 6/50
632/632 [==============================] - 32s 51ms/step - loss: 0.0644 - accuracy: 0.9841 - val_loss: 4.0687 - val_accuracy: 0.4460
Epoch 7/50
632/632 [==============================] - 32s 51ms/step - loss: 0.0370 - accuracy: 0.9897 - val_loss: 4.7731 - val_accuracy: